<a href="https://colab.research.google.com/github/fredericpoitevin/tseries-deconvolution/blob/master/Deconvolution_of_Michael_coronavirus_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

We are interested in the following scenario:
We have access to two cumulative time-series that we call cause (or _cases_ $C(t)$ the total number of cases at time $t$) and consequence (or _deaths_ $D(t) $ the total number of deaths at time $t$). We assume that consequence is cause convolved with an unknown kernel $K$, and we are interested in inferring the kernel function, or at least its parameters.

Formally, 

$$\frac{\partial D}{\partial \tau}|_{\tau=t} = \sum_{s=0}^{t} K(t-s)\frac{\partial C}{\partial \tau}|_{\tau=s}$$

We have access to samples of the cumulative time-series at fixed time intervals $\Delta t$, we can thus discretize the above equation with finite difference. For $i \geq 1$, we introduce $f_{i} = \frac{F(i\Delta t) - F((i-1)\Delta t)}{\Delta t}$:

$$d_{i} = \sum_{j=1}^{i}k_{ij}c_{j}$$

In [0]:
|

# Toolkit

In [0]:
# load all libraries
import os
import numpy as np
from matplotlib import pyplot as plt
from google.colab import files
from scipy.optimize import curve_fit

In [0]:
def synthesize_tseries(t, function='gaussian', params=None, normalize=False):
  """synthesize_tseries(t, function='gaussian', params=None, normalize=False)
  Arguments:
  ---------
  - t: time-points (array. Mandatory)
  - function: shape of the distribution (only 'gaussian' is available for now. Optional, gaussian by default)
  - params: parameters of function (optional or mandatory depending on function)
  - normalize: whether function is to be normalized or not.
  Returns:
  -------
  tseries: function evaluated at t
  """
  if(function=='gaussian'):
    if params is None:
      print('WARNING! please provide the following parameters for your gaussian function')
      print('>>>>>>>> gaussian = synthesize_tseries(t, params=[amplitude, origin, width])')
    else:
      tseries = gaussian(t, params[0], params[1], params[2], normalize=normalize)
  else:
    print('WARNING! only gaussian function has been implemented so far...')
    tseries=None
  return tseries

def gaussian(x, amplitude, center, sigma, normalize=False):
  """gaussian(x, amplitude, center, sigma)
  Arguments:
  - x         : data points at which gaussian is evaluated (mandatory)
  - amplitude : mandatory
  - center    : mandatory
  - sigma     : mandatory
  - normalize : if True, rescales amplitude to normalize gaussian
  """
  if normalize:
    amplitude /= np.sqrt(2*np.pi)*sigma
  return amplitude*np.exp(-0.5*((x-center)/sigma)**2)